In [111]:
import pandas as pd # para carregar o CSV
import numpy as np

from sklearn import tree
from sklearn.datasets import load_boston
from sklearn.svm import SVR
from sklearn.metrics import r2_score

In [7]:
X = [[0, 0], [2, 2]]
y = [0.5, 2.5]
clf = tree.DecisionTreeRegressor()
clf = clf.fit(X, y)
clf.predict([[1, 1]])

array([0.5])

In [8]:
print(clf.predict([[1, 1]]))

[0.5]


In [35]:
sinais_vitais = pd.read_csv(r'sinais_vitais.txt')
sinais_vitais.set_index('id', inplace=True)

In [46]:
sinais_vitais.head()
sinais = sinais_vitais[['qPA','pulso', 'resp']].values
gravidade = sinais_vitais['grav'].values

array([40.      , 41.530427, 52.730745, ..., 24.530704, 61.062546,
       39.110506])

In [62]:
sinais_p1 = sinais[:1300]
grav_p1 = gravidade[:1300]

In [103]:
clf = tree.DecisionTreeRegressor()

predictions = []
#clf = clf.fit(sinais_p1, grav_p1)
#clf.predict([[4.745181,196.803149,11.875068],[1.707746,184.267283,14.256125]])

In [104]:
#k_fold com k=3 , utilizando a primeira e segunda parte
div = 3
sinais_k = np.array_split(sinais, div)
grav_k = np.array_split(gravidade, div)
clf = tree.DecisionTreeRegressor()
clf = clf.fit(np.concatenate((sinais_k[0], sinais_k[1])), np.concatenate((grav_k[0], grav_k[1])))
prediction = clf.predict(sinais_k[2])

predictions.append(r2_score(grav_k[2], prediction))

In [105]:
#k_fold com k=3 , utilizando a primeira e ultima parte
#testando com id=549 e id=550
clf = tree.DecisionTreeRegressor()
clf = clf.fit(np.concatenate((sinais_k[0], sinais_k[2])), np.concatenate((grav_k[0], grav_k[2])))
prediction = clf.predict(sinais_k[1])

predictions.append(r2_score(grav_k[1], prediction))

In [106]:
#k_fold com k=3 , utilizando a segunda e ultima parte
clf = tree.DecisionTreeRegressor()
clf = clf.fit(np.concatenate((sinais_k[1], sinais_k[2])), np.concatenate((grav_k[1], grav_k[2])))
prediction = clf.predict(sinais_k[0])

predictions.append(r2_score(grav_k[0], prediction))

In [107]:
predictions

[0.9436134990478889, 0.9378724784883554, 0.9150248145198612]

In [108]:
# deve ter um nivel de aleatoriedade no algoritmo porque ele dá diferentes valores
np.mean(predictions)

0.9321702640187018